In [1]:
import numpy as np
import math
from Crypto.Util import number
import Crypto.Random as Random

In [2]:
CATEGORIES = [0, 1, 2, 3, 4]

def buildParams(epsilon):
    d = len(CATEGORIES)
    l, n = decideRatio(epsilon, d)
    assert (n-l) % d == 0, "Invalied combination, n, l, d"
    print("n: ", n)
    print("l: ", l)
    D = max([l, (n-l)//d]) + 1
    return d, l, n, D

def decideRatio(eps, d):
    ratio = np.exp(eps) / ((d-1) + np.exp(eps))
    integer = int(ratio * 1000)
    while integer > 0:
        if (1000-integer) % d == 0:
            g = math.gcd(integer, 1000, (1000-integer) // d)
            return integer // g, 1000 // g
        integer -= 1
    assert False, "Not found"    

class R:
    def __init__(self, data, epsilon):
        self.data = data
        self.epsilon = epsilon
        self.d, self.l, self.n, self.D = buildParams(epsilon)
        
    def setup(self):
        t = self.data
        print("secret input: ", t)
        mu_array = []
        mu_array = [t]*self.l
        for category in CATEGORIES:
            mu_array = mu_array + ([category] * ((self.n - self.l) // self.d))
        mu_array = np.random.permutation(mu_array).tolist()
        self.mu_array = mu_array
        print("mu array: ", mu_array)
        
    def setPubKey(self, pub_key):
        self.pub_key = pub_key
    
    def step2(self, g_a, g_b, g_ab):
        q, g, h = self.pub_key
        w_array = []
        y_array = []
        v_array = []
        
        for i in range(1, self.n+1):
            r = number.getRandomRange(1, q-1)
            s = number.getRandomRange(1, q-1)
            w = pow(g, r, q) * pow(g_a, s, q) % q
            v = pow(g_b, r, q) * pow(g_ab * pow(g, i-1, q) % q, s, q) % q
            y = pow(g, self.D**self.mu_array[i-1], q) * pow(h, v, q) % q
            w_array.append(w)
            y_array.append(y)
            v_array.append(v)

        self.w_array = w_array
        self.y_array = y_array
        self.v_array = v_array

        return v_array, w_array, y_array
    
    def AKEncBool1(self):
        q, g, h = self.pub_key
        b_array = []
        c_array = []
        s_array = []
        random_w_array = []
        for y, mu in zip(self.y_array, self.mu_array):
            random_w = number.getRandomRange(1, q-1)
            c = [0] * self.d
            s = [0] * self.d
            b = [0] * self.d
            for category in CATEGORIES:
                if category != mu:
                    c_i = number.getRandomRange(1, q-1)
                    s_i = number.getRandomRange(1, q-1)
                    c[category] = c_i
                    s[category] = s_i
                    g_i_inv = pow(pow(g, self.D**category, q), -1, q)
                    deno = pow(y * g_i_inv % q, c_i, q)
                    b_i = pow(h, s_i, q) * pow(deno, -1, q) % q
                    b[category] = b_i
                else:
                    b_i = pow(h, random_w, q)
                    b[category] = b_i
            random_w_array.append(random_w)
            b_array.append(b)
            c_array.append(c)
            s_array.append(s)

        self.b_array = b_array
        self.s_array = s_array
        self.c_array = c_array
        self.random_w_array = random_w_array
        
        return b_array
    
    def AKEncBool3(self, ak_enc_bool_x_array):
        q, g, h = self.pub_key
        for x,c,s,mu,v,random_w in zip(ak_enc_bool_x_array, self.c_array, self.s_array, self.mu_array, self.v_array, self.random_w_array):
            for category in CATEGORIES:
                if category == mu:
                    c[mu] = x - sum(c)
                    s[mu] = v * c[mu] + random_w

        return self.c_array, self.s_array
    
    def AKLin1(self):
        q, g, h = self.pub_key
        b_lin = [0] * self.d
        c_lin = [0] * self.d
        s_lin = [0] * self.d
        random_w = number.getRandomRange(1, q-1)
        common = sum([self.D**category for category in CATEGORIES]) * ((self.n - self.l) // self.d)
        for category in CATEGORIES:
            if category != self.data:
                total = common + self.l * self.D**category
                c_i = number.getRandomRange(1, q-1)
                s_i = number.getRandomRange(1, q-1)
                c_lin[category] = c_i
                s_lin[category] = s_i
                g_i_inv = pow(pow(g, total, q), -1, q)
                commitment = 1
                for y in self.y_array:
                    commitment = commitment * y % q
                deno = pow(commitment * g_i_inv % q, c_i, q)
                b_i = pow(h, s_i, q) * pow(deno, -1, q) % q
                b_lin[category] = b_i
            else:
                b_i = pow(h, random_w, q)
                b_lin[category] = b_i

        self.b_lin = b_lin
        self.s_lin = s_lin
        self.c_lin = c_lin
        self.random_w_lin = random_w
        return b_lin

    def AKLin3(self, ak_enc_bool_x_lin):
        q, g, h = self.pub_key
        for category in CATEGORIES:
            if category == self.data:
                self.c_lin[self.data] = ak_enc_bool_x_lin - sum(self.c_lin)
                v_sum = 0
                for v in self.v_array:
                    v_sum += v
                self.s_lin[self.data] = v_sum * self.c_lin[self.data] + self.random_w_lin
        return self.c_lin, self.s_lin
    
class I:
    def __init__(self, epsilon):
        self.epsilon = epsilon
        self.d, self.l, self.n, self.D = buildParams(epsilon)
    
    def setup(self, security):
        q = number.getPrime(2 * security, Random.new().read)        
        g = number.getRandomRange(1, q-1)
        h = number.getRandomRange(1, q-1)
        
        self.q = q
        self.g = g
        self.h = h
        
        self.sigma = np.random.randint(1, self.n+1)
        print("sigma: ", self.sigma)
        
        self.pub_key = (q, g, h)
        
    def step1(self):
        a = number.getRandomRange(1, self.q-1)
        b = number.getRandomRange(1, self.q-1)
        self.a = a
        self.b = b
        
        g_a = pow(self.g, a, self.q)
        g_b = pow(self.g, b, self.q)
        g_ab = pow(self.g, a * b - self.sigma + 1, self.q)

        return g_a, g_b, g_ab
        
    def step3(self, w_array, y_array):
        v_sigma = pow(w_array[self.sigma-1], self.b, self.q)
        g_mu_sigma = y_array[self.sigma-1] * pow(pow(self.h, v_sigma, self.q), -1, self.q) % self.q
        secret_output = None
        for category in CATEGORIES:
            if pow(self.g, self.D**category, self.q) == g_mu_sigma:
                secret_output = category
        print("secret output: ", secret_output, "g^{mu_sigma}: ", g_mu_sigma)
        return secret_output
    
    def AKEncBool2(self, num):
        self.ak_enc_bool_x_array = []
        for _ in range(num):
            self.ak_enc_bool_x_array.append(number.getRandomRange(1, self.q-1))
        return self.ak_enc_bool_x_array

    def AKEncBool4(self, s_array, c_array, b_array, y_array):
        print("####### AKEncBool verification #######")
        for s,c,b,y,x in zip(s_array, c_array, b_array, y_array, self.ak_enc_bool_x_array):
            for i in CATEGORIES:
                if pow(self.h, s[i], self.q) != b[i] * pow(y * pow(pow(self.g, self.D**i, self.q), -1, self.q) % self.q, c[i], self.q) % self.q:
                    print("AKEncBool False1.")
                    return False
            if x != sum(c):
                print("AKEncBool False2.")
                return False
        print("AKEncBool OK.")
        return True 
    
    def AKLin2(self):
        self.ak_enc_bool_x_lin = 0
        self.ak_enc_bool_x_lin = number.getRandomRange(1, self.q-1)
        return self.ak_enc_bool_x_lin
    
    def AKLin4(self, s_lin, c_lin, b_lin, y_array):
        print("####### AKLin verification #######")
        common = sum([self.D**category for category in CATEGORIES]) * ((self.n - self.l) // self.d)
        commitment = 1
        for y in y_array:
            commitment = commitment * y % self.q
        for category in CATEGORIES:
            total = common + self.l * self.D**category
            if pow(self.h, s_lin[category], self.q) != b_lin[category] * pow(commitment * pow(pow(self.g, total, self.q), -1, self.q) % self.q, c_lin[category], self.q) % self.q:
                print("AKEncBool False1.")
                return False
        if self.ak_enc_bool_x_lin != sum(c_lin):
            print("AKEncBool False2.")
            return False
        print("AKEncBool OK.")
        return True 

In [3]:
epsilon = 1.0
secret_input = 0
sender = R(secret_input, epsilon)
sender.setup()

receiver = I(epsilon)
receiver.setup(security=80)

pub_key = receiver.pub_key
sender.setPubKey(pub_key)

g_a, g_b, g_ab = receiver.step1()
v_array, w_array, y_array = sender.step2(g_a, g_b, g_ab)
x = receiver.step3(w_array, y_array)

b_array = sender.AKEncBool1()
ak_enc_bool_x_array = receiver.AKEncBool2(len(y_array))
c_array, s_array = sender.AKEncBool3(ak_enc_bool_x_array)
receiver.AKEncBool4(s_array, c_array, b_array, y_array)

b_lin = sender.AKLin1()
ak_enc_bool_x_lin = receiver.AKLin2()
c_lin, s_lin = sender.AKLin3(ak_enc_bool_x_lin)
receiver.AKLin4(s_lin, c_lin, b_lin, y_array)

n:  25
l:  10
secret input:  0
mu array:  [0, 4, 0, 0, 3, 3, 0, 0, 0, 1, 4, 1, 2, 0, 0, 2, 0, 1, 0, 0, 2, 4, 0, 3, 0]
n:  25
l:  10
sigma:  18
secret output:  1 g^{mu_sigma}:  567682780190953027376887203412270526562880420346
####### AKEncBool verification #######
AKEncBool OK.
####### AKLin verification #######
AKEncBool OK.


True

In [ ]:
x_list = []
for i in range(100):
    epsilon = 1.0
    secret_input = 1
    sender = R(secret_input, epsilon)
    sender.setup()

    receiver = I(epsilon)
    receiver.setup(security=80)

    pub_key = receiver.pub_key
    sender.setPubKey(pub_key)

    g_a, g_b, g_ab = receiver.step1()
    v_array, w_array, y_array = sender.step2(g_a, g_b, g_ab)
    x_list.append(receiver.step3(w_array, y_array))

In [ ]:
import collections
collections.Counter(x_list)